# A/B-тестирование

## Задача 

Провести оценку результатов A/B-теста. В нашем распоряжении есть датасет с действиями пользователей, техническое задание и несколько вспомогательных датасетов.

## Цель

Оценить корректность проведения теста и проанализировать его результаты:
- удостовериться, что нет пересечений с конкурирующим тестом и нет пользователей, участвующих в двух группах теста одновременно;
- проверить равномерность распределения пользователей по тестовым группам и правильность их формирования.

In [1]:
pip install -U kaleido

In [2]:
# импортируем библиотеки, которые нам понадобятся в дальнейшем
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import pandas as pd
import numpy as np
from IPython.display import display
import warnings
import scipy.stats as stats
from datetime import datetime, timedelta
import plotly.express as px
from plotly import graph_objects as go
from statsmodels.stats.weightstats import ztest as ztest
pd.options.mode.chained_assignment = None

In [3]:
try:
    fa_events = pd.read_csv('/datasets/final_ab_events.csv', sep=',')
    fa_participants = pd.read_csv('/datasets/final_ab_participants.csv', sep=',')
    fa_new_users = pd.read_csv('/datasets/final_ab_new_users.csv', sep=',')
    apm_events = pd.read_csv('/datasets/ab_project_marketing_events.csv', sep=',')
except:  
    fa_events = pd.read_csv(r'D:\Python\final_ab_events.csv', sep=',')
    fa_participants = pd.read_csv(r'D:\Python\final_ab_participants.csv', sep=',')
    fa_new_users = pd.read_csv(r'D:\Python\final_ab_new_users.csv', sep=',')
    apm_events = pd.read_csv(r'D:\Python\ab_project_marketing_events.csv', sep=',')

## Предобработка данных

In [4]:
# создадим функцию для удобства просмотра информации о датасетах
def info_df(i):
    display(i.head()),
    i.info(),
    display(i.describe()),
    display(i.value_counts()),
    print('Всего дубликатов: ', i.duplicated().sum()),
    print('Всего пропусков: ', i.isna().sum())

In [5]:
info_df(fa_events)

,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     440317 non-null  object 
 1   event_dt    440317 non-null  object 
 2   event_name  440317 non-null  object 
 3   details     62740 non-null   float64
dtypes: float64(1), object(3)
memory usage: 13.4+ MB


,details
count,62740.000000
mean,23.877631
std,72.180465
min,4.990000
25%,4.990000
50%,4.990000
75%,9.990000
max,499.990000


user_id           event_dt             event_name  details
000199F1887AE5E6  2020-12-14 09:56:09  purchase    4.99       1
A9ED1B6A4C9AE26A  2020-12-29 12:26:50  purchase    4.99       1
A9EE24521E7EACC1  2020-12-12 18:56:05  purchase    9.99       1
                  2020-12-13 12:12:30  purchase    4.99       1
                  2020-12-14 02:12:41  purchase    4.99       1
                                                             ..
53C62312FEE05432  2020-12-22 14:12:41  purchase    9.99       1
                  2020-12-25 10:57:28  purchase    4.99       1
                  2020-12-27 08:25:36  purchase    4.99       1
53C7EB754EB9F9C4  2020-12-17 11:36:36  purchase    4.99       1
FFF8FDBE2FE99C91  2020-12-23 02:40:19  purchase    4.99       1
Length: 62740, dtype: int64

Всего дубликатов:  0
Всего пропусков:  user_id            0
event_dt           0
event_name         0
details       377577
dtype: int64


In [6]:
info_df(fa_participants)

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18268 entries, 0 to 18267
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  18268 non-null  object
 1   group    18268 non-null  object
 2   ab_test  18268 non-null  object
dtypes: object(3)
memory usage: 428.3+ KB


,user_id,group,ab_test
count,18268,18268,18268
unique,16666,2,2
top,0FDFDA0B2DEC2D91,A,interface_eu_test
freq,2,9655,11567


user_id           group  ab_test                
0002CE61FF2C4011  A      interface_eu_test          1
AA302D044EB2BED4  B      recommender_system_test    1
AA55A2BDAE39BECC  B      recommender_system_test    1
AA559B5A4F3F5E61  A      recommender_system_test    1
AA49424D3F80982D  A      interface_eu_test          1
                                                   ..
564A51795FD96FEA  A      interface_eu_test          1
564B9FE4D8A7B2BA  B      interface_eu_test          1
564F31145C1EEDB4  A      interface_eu_test          1
5650B53B91DC98BB  A      interface_eu_test          1
FFF58BC33966EB51  B      interface_eu_test          1
Length: 18268, dtype: int64

Всего дубликатов:  0
Всего пропусков:  user_id    0
group      0
ab_test    0
dtype: int64


In [7]:
info_df(fa_new_users)

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     61733 non-null  object
 1   first_date  61733 non-null  object
 2   region      61733 non-null  object
 3   device      61733 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB


,user_id,first_date,region,device
count,61733,61733,61733,61733
unique,61733,17,4,4
top,D72A72121175D8BE,2020-12-21,EU,Android
freq,1,6290,46270,27520


user_id           first_date  region     device 
0001710F4DDB1D1B  2020-12-14  EU         Android    1
AA1FF3A07635117C  2020-12-11  EU         PC         1
AA4281556A9ECA98  2020-12-09  EU         Mac        1
AA43F0C37A670E3B  2020-12-09  EU         Mac        1
AA47A433929EE0CD  2020-12-20  N.America  Android    1
                                                   ..
5543CE8C1906370E  2020-12-15  CIS        iPhone     1
5545492A601FE695  2020-12-09  EU         iPhone     1
5545992A45085197  2020-12-11  APAC       PC         1
554644904BBE54A7  2020-12-12  EU         iPhone     1
FFFFE36C0F6E92DF  2020-12-22  N.America  Mac        1
Length: 61733, dtype: int64

Всего дубликатов:  0
Всего пропусков:  user_id       0
first_date    0
region        0
device        0
dtype: int64


In [8]:
info_df(apm_events)

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 580.0+ bytes


,name,regions,start_dt,finish_dt
count,14,14,14,14
unique,14,6,14,14
top,Christmas&New Year Promo,APAC,2020-12-25,2021-01-03
freq,1,4,1,1


name                              regions                   start_dt    finish_dt 
4th of July Promo                 N.America                 2020-07-04  2020-07-11    1
Black Friday Ads Campaign         EU, CIS, APAC, N.America  2020-11-26  2020-12-01    1
CIS New Year Gift Lottery         CIS                       2020-12-30  2021-01-07    1
Chinese Moon Festival             APAC                      2020-10-01  2020-10-07    1
Chinese New Year Promo            APAC                      2020-01-25  2020-02-07    1
Christmas&New Year Promo          EU, N.America             2020-12-25  2021-01-03    1
Dragon Boat Festival Giveaway     APAC                      2020-06-25  2020-07-01    1
Easter Promo                      EU, CIS, APAC, N.America  2020-04-12  2020-04-19    1
International Women's Day Promo   EU, CIS, APAC             2020-03-08  2020-03-10    1
Labor day (May 1st) Ads Campaign  EU, CIS, APAC             2020-05-01  2020-05-03    1
Single's Day Gift Promo           APA

Всего дубликатов:  0
Всего пропусков:  name         0
regions      0
start_dt     0
finish_dt    0
dtype: int64


**Подвывод:**

- Cтолбцы с датой нужно привести к нужному типу datetime
- Cтолбец details датасета fa_events содержит в себе множество пропусков
- Дубликаты в данных не выявлены

In [9]:
# посмотрим, что именно в столбце с пропусками 
fa_events['details'].value_counts()

4.99      46362
9.99       9530
99.99      5631
499.99     1217
Name: details, dtype: int64

В столбце с пропусками обнаружены детали покупок, которые не несут в себе важной информации для исследования, поэтому на них не будем акцентировать внимание, т.к. некоторые покупки могут быть и без дополнительной информации и, следовательно, там и будет пропуск в этом столбце 

In [10]:
# Cтолбцы с датой нужно приведем к нужному типу datetime
apm_events['start_dt'] = pd.to_datetime(apm_events['start_dt'])
apm_events['finish_dt'] = pd.to_datetime(apm_events['finish_dt'])
fa_new_users['first_date'] = pd.to_datetime(fa_new_users['first_date'])
fa_events['event_dt'] = pd.to_datetime(fa_events['event_dt'])

In [11]:
fa_participants.head()

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test


## Оценим корректность проведения теста

In [12]:
fa_participants

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test
...,...,...,...
18263,1D302F8688B91781,B,interface_eu_test
18264,3DE51B726983B657,A,interface_eu_test
18265,F501F79D332BE86C,A,interface_eu_test
18266,63FBE257B05F2245,A,interface_eu_test


In [13]:
# посмотрим распределение тестов 
fa_participants['ab_test'].value_counts()

interface_eu_test          11567
recommender_system_test     6701
Name: ab_test, dtype: int64

In [14]:
# Посмотрим кол-во участников
n = fa_participants['user_id'].nunique()
print('Всего уникальных участников теста: ', n)

Всего уникальных участников теста:  16666


In [15]:
recommender_system_test = fa_participants.query('ab_test == "recommender_system_test"')

In [16]:
recommender_system_test['ab_test'].value_counts()

recommender_system_test    6701
Name: ab_test, dtype: int64

In [17]:
# Посмотрим кол-во участников
n_system_test = recommender_system_test['user_id'].nunique()
print('Всего участников теста recommender_system_test: ', n_system_test)

Всего участников теста recommender_system_test:  6701


В задании у нас ожидаемое количество участников теста: 6000. Получилось немного больше, но это некритично. 

In [18]:
max_date = fa_new_users['first_date'].max()
print('Посмотрим на максимальную дату, зарегистрировавшихся пользователей: ', max_date)

Посмотрим на максимальную дату, зарегистрировавшихся пользователей:  2020-12-23 00:00:00


In [19]:
rec_sys_test = fa_participants.query('ab_test == "recommender_system_test"')
rec_sys_test

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test
...,...,...,...
6696,053FB26D6D49EDDC,A,recommender_system_test
6697,9D263B8EF15CF188,B,recommender_system_test
6698,F2FBBA33F37DEC46,A,recommender_system_test
6699,29C92313A98B1176,B,recommender_system_test


In [20]:
for_max_date = rec_sys_test.merge(fa_new_users, how='left', on='user_id')
for_max_date

,user_id,group,ab_test,first_date,region,device
0,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC
1,A7A3664BD6242119,A,recommender_system_test,2020-12-20,EU,iPhone
2,DABC14FDDFADD29E,A,recommender_system_test,2020-12-08,EU,Mac
3,04988C5DF189632E,A,recommender_system_test,2020-12-14,EU,iPhone
4,482F14783456D21B,B,recommender_system_test,2020-12-14,EU,PC
...,...,...,...,...,...,...
6696,053FB26D6D49EDDC,A,recommender_system_test,2020-12-10,N.America,Android
6697,9D263B8EF15CF188,B,recommender_system_test,2020-12-16,N.America,Mac
6698,F2FBBA33F37DEC46,A,recommender_system_test,2020-12-18,APAC,Mac
6699,29C92313A98B1176,B,recommender_system_test,2020-12-07,APAC,Android


In [21]:
new_max_date = for_max_date['first_date'].max()
new_max_date

Timestamp('2020-12-21 00:00:00')

In [22]:
print('Посмотрим на максимальную дату, зарегистрировавшихся пользователей теста "recommender_system_test": ', new_max_date)

Посмотрим на максимальную дату, зарегистрировавшихся пользователей теста "recommender_system_test":  2020-12-21 00:00:00


Максимальная дата зарегистрировавшихся пользователей теста совпадает с ТЗ - здесь все нормально

In [23]:
# выделим необходимыe данные для дальнейшего исследования
for_abtest = fa_new_users[fa_new_users['user_id'].isin(fa_participants.query('ab_test == "recommender_system_test"')['user_id'])]
for_abtest.head()

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
13,E6DE857AFBDC6102,2020-12-07,EU,PC
20,DD4352CDCF8C3D57,2020-12-07,EU,Android
23,831887FE7F2D6CBA,2020-12-07,EU,Android
39,4CB179C7F847320B,2020-12-07,EU,iPhone


In [24]:
begin_test = for_abtest['first_date'].min()
print('Дата начала набора новых пользователей для теста: ', begin_test)

Дата начала набора новых пользователей для теста:  2020-12-07 00:00:00


Теперь все даты сходятся - дата запуска: 2020-12-07 и дата остановки набора новых пользователей: 2020-12-21 

In [25]:
last = fa_events['event_dt'].max()
print('Дата последнего события: ', last)

Дата последнего события:  2020-12-30 23:36:33


По ТЗ дата остановки: 2021-01-04

Далее нужно узнать, что в тест должно быть отобрано 15% новых пользователей из региона EU

In [28]:
for_abtest['region'].value_counts()

EU           6351
N.America     223
APAC           72
CIS            55
Name: region, dtype: int64

In [ ]:
percent = round((for_abtest.query('region == "EU"')['region'].count()/new_max_date.query('region == "EU"')['region'].count())*100, 2)
print('Всего пользователей из EU региона в процентах:', percent)

Здесь все совпадает с данными из ТЗ

## Время проведения теста

Убедимся, что время не совпадает с маркетинговыми и другими активностями.

In [ ]:
apm_events.query('"2020-12-07" < start_dt < "2021-01-04"')

В период проведения теста проходят 2 акции: 
- Christmas&New Year Promo - может оказать существенное влияние, потому что она затрагивает период проведения теста и идет с 25 декабря по 3 января и куда более информативно - это регион, в котором она проходит - EU, который преобладает в тесте;
- CIS New Year Gift Lottery - окажет малозаметное влияние, так как регион ее проведения CIS и процент пользователей из этого региона, участвовавших в тесте, совсем незначительный

## Аудитория теста

Здесь удостоверимся, что нет пересечений с конкурирующим тестом и нет пользователей, участвующих в двух группах теста одновременно. Проверим равномерность распределения по тестовым группам и правильность их формирования.

In [ ]:
# определим есть ли пользователи, входящие в оба теста сразу
n_ab_test = fa_participants.groupby('user_id').agg({'ab_test': 'nunique'}).sort_values(by = 'ab_test', ascending = False).reset_index()
bi = n_ab_test[n_ab_test['ab_test'] == 2]
b = len(bi)
print('Всего пользователей в обоих тестах одновременно:', b)

In [ ]:
# посмотрим, в каких группах состоят полшьзователи
nusers_in_tests = bi['user_id']
nusers_in_tests = fa_participants.query('user_id in @nusers_in_tests').reset_index(drop=True)
nusers_in_tests.groupby(['ab_test', 'group']).agg({'user_id': 'nunique'})

In [ ]:
# посмотрим равномерно ли распределены пользователи в группе В
ob_normal_B = fa_participants[fa_participants['user_id'].duplicated()].query('group == "B"')['user_id']
(fa_participants[fa_participants['user_id'].isin(ob_normal_B)].query('ab_test == "recommender_system_test"')['group'].value_counts())

В итоге мы увидели, что распределение пользователей группы B теста interface_eu_test в тесте recommender_system_test довольно равномерное. 

In [ ]:
# теперь перепроверим присутствуют ли пользователи, входящие в обе группы А и В 
fa_participants.query('ab_test == "recommender_system_test"')['user_id'].duplicated().sum()

Для дальнейшей работы стоит объединить датасеты

In [ ]:
for_abtest_users = for_abtest.merge(fa_participants.query('ab_test == "recommender_system_test"'), on='user_id')
display(for_abtest_users.user_id.nunique())


In [ ]:
for_abtest_users = for_abtest_users.merge(fa_events, how='left', on='user_id')
display(for_abtest_users.nunique())

Теперь нужно отфильтровать таблицу и обозначить дни, которые нам понадобятся в дальнейшем

In [ ]:
horizon_days = 14
observation_date = datetime(2021, 1, 4).date()
last_suitable_acquisition_date = observation_date - timedelta(days = horizon_days - 1)

In [ ]:
registration_test = for_abtest_users
for_abtest_users = for_abtest_users.query('event_dt <= @last_suitable_acquisition_date')
display(for_abtest_users.user_id.nunique())

In [ ]:
for_abtest_users['lifetime'] = (for_abtest_users['event_dt'] - for_abtest_users['first_date']).dt.days
for_abtest_users = for_abtest_users.query('lifetime < 14')
display(for_abtest_users.user_id.nunique())

## Исследовательский анализ данных

### Кол-во событий на пользователя в выборках

In [ ]:
registration_test.head()

In [ ]:
# сделаем срез пользователей, которые прошли регистрацию
after_registration_test = registration_test[registration_test['event_dt'].isna()]
after_registration_test = (registration_test.groupby(['first_date','group'], as_index=False)['user_id'].nunique()
                            .rename(columns={'user_id':'n_users'}))
after_registration_test

Теперь посмотрим кол-во событий по пользователями и событиям  

In [ ]:
u_with_events = (for_abtest_users.groupby(['user_id', 'event_name'], as_index=False)['region'].count()
                              .rename(columns={'region':'n_events'})
                              .merge(for_abtest_users[['user_id', 'group']], on='user_id')
                              .drop_duplicates())
u_with_events.head()

Посмотрим кол-во событий по группам

In [ ]:
u_with_events_gruppen = (u_with_events.groupby(['group', 'event_name'], as_index=False)['n_events'].agg('sum'))
u_with_events_gruppen['event_with_users'] = 0
u_with_events_gruppen

Посмотрим кол-во оставшихся в тесте пользователей в группах А и В

In [ ]:
a = for_abtest_users.query('group == "A"')['user_id'].nunique()
b = for_abtest_users.query('group == "B"')['user_id'].nunique()
print('Всего пользователей из группы а:',a)
print('Всего пользователей из группы b:',b)

Далее нужно определить кол-во событий на одного пользователя 

In [ ]:
a2 = u_with_events_gruppen.loc[u_with_events_gruppen['group'] == 'A', 'event_with_users'] = (u_with_events_gruppen['n_events'] / a)

b2 = u_with_events_gruppen.loc[u_with_events_gruppen['group'] == 'B', 'event_with_users'] = (u_with_events_gruppen['n_events'] / b)

In [ ]:
u_with_events_gruppen

In [ ]:
# посмотрим наглядно это на графике
fig = px.box(u_with_events, x="event_name", y="n_events", color="group",
             title="Количество событий на одного пользователя" 
             )
fig.update_xaxes(title_text='Событие')
fig.update_yaxes(title_text='Кол-во событий')
fig.show()

По графику можно заключить, что среднее кол-во событий на одного клиента практически не различается между группами 


### Распределение числа событий в выборках по дням

In [ ]:
for_abtest_users.head()

In [ ]:
for_abtest_users.info()

In [ ]:
# оставим в столбце event_dt только даты
for_abtest_users['event_dt'] = for_abtest_users['event_dt'].dt.date
for_abtest_users.head()

In [ ]:
for_abtest_users.info()

In [ ]:
for_abtest_users['event_dt'] = pd.to_datetime(for_abtest_users['event_dt'])

In [ ]:

dt_with_event = (for_abtest_users.groupby(['event_dt', 'group'], as_index=False)['user_id'].count().rename(columns={'user_id':'n_events'}))
dt_with_event.head()

In [ ]:
# посмотрим это все на графике
fig = px.bar(dt_with_event, x='event_dt', y='n_events', color='group', 
             title='Распределение числа событий в выборках по дням' 
             )
fig.update_xaxes(title_text='Дата')
fig.update_yaxes(title_text='Кол-во событий')

fig.show()

По графику видно:
- Пик событий 21 декабря и в основном это у группы А;
- Группа В превосходит группу А только 7 и 9 декабря, а в оставшиеся дни группа А уверенно опережала группу В по кол-ву событий;
- Начиная c 14 декабря можно наблюдать резкий рост кол-ва событий у группы А.

Посмотрим на динамику набора пользователей - возможно удастся выявить взаимосвязь и выяснить, от чего произошел столь сильный скачок 14 декабря

In [ ]:
registrations_users = (for_abtest_users.groupby(['event_dt', 'group'], as_index=False)['user_id'].nunique().rename(columns={'user_id':'n_users'}))
registrations_users.head()

In [ ]:
fig = px.bar(registrations_users, x='event_dt', y='n_users', color='group', 
             title='Динамика набора пользователей по дням'
             )
fig.update_xaxes(title_text='Дата')
fig.update_yaxes(title_text='Кол-во пользователей')

fig.show()

Вот и удалось выявить взаимосвязь - 14 декабря также произошел резкий скачок в наборе новых пользователей и, следовательно, они совершали какие-либо действия(события). Тем самым это зеркально отразилось и на графике в предыдущем пункте исследования

### Kонверсия в воронке в выборках на разных этапах

In [ ]:
# создадим переменные для иллюстрации воронки
funl_for_grA = for_abtest_users[for_abtest_users['group']=="A"].groupby('event_name').agg({'event_name':'count', 'user_id':'nunique'})
funl_for_grA.columns = ['n_event','n_user']
funl_for_grA = funl_for_grA.sort_values(by = 'n_event', ascending = False).reset_index().reindex([0,1,3,2])
funl_for_grA


In [ ]:
funl_for_grB = for_abtest_users[for_abtest_users['group']=="B"].groupby('event_name').agg({'event_name':'count', 'user_id':'nunique'})
funl_for_grB.columns = ['n_event','n_user']
funl_for_grB = funl_for_grB.sort_values(by = 'n_event', ascending = False).reset_index()
funl_for_grB

Построим воронку конверсий в выборках на разных этапах

In [ ]:
fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'A',
    y = funl_for_grA['event_name'],
    x = funl_for_grA['n_user'],
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = 'B',
    orientation = "h",
    y = funl_for_grB['event_name'],
    x = funl_for_grB['n_user'],
    textposition = "inside",
    textinfo = "value+percent previous"))


fig.update_layout(title_text = 'Bоронкa конверсий в выборках на разных этапах')
fig.show()

- Наибольшая просадка коверсии между login и product_page уровнями - (-35% у группы А и -44% у группы В). 
- До покупки доходят 30% пользователей группы А и 28% пользователей группы В. 
- Наименьшая разница в конверсии между product_cart и purchase - (разница в 2% у группы А, у группы В разница и вовсе отсутствует), это свидетельствует о том, что пользователи предпочитают после добавления товара в корзину практически всегда его покупать. 

### Оценка результата A/B-тестирования

In [ ]:
for_abtest_users.head()

In [ ]:
#создадим датасет по столбцам 'lifetime', 'group', 'event_name' и с кол-вом событий
n_events_lifetime = (for_abtest_users.groupby(['lifetime', 'group', 'event_name'], as_index=False)['user_id'].count()
                  .rename(columns={'user_id':'n_events'}))
n_events_lifetime.head()

Дальше нужно создать датафрейм со страницей выбора товара и расчетом совокупной суммы событий

In [ ]:
prod_page = n_events_lifetime.query('event_name == "product_page"')
prod_page['cumsum_n_events'] = prod_page.groupby('group')['n_events'].cumsum()
prod_page.head()


In [ ]:
# расчитаем кол-во событий за 14 дней лайфтайма
prod_page_A = round((prod_page.iloc[-2]['cumsum_n_events'] - prod_page.iloc[0]['cumsum_n_events']) / prod_page.iloc[0]['cumsum_n_events'], 2) 
prod_page_A


In [ ]:
# делаем тоже самое для второй группы
prod_page_B = round((prod_page.iloc[-1]['cumsum_n_events'] - prod_page.iloc[1]['cumsum_n_events']) / prod_page.iloc[1]['cumsum_n_events'], 2) 
prod_page_B

Теперь проделаем тоже самое с другим событием - "добавлением товара в корзину" 

In [ ]:
prod_cart = n_events_lifetime.query('event_name == "product_cart"')
prod_cart['cumsum_n_events'] = prod_cart.groupby('group')['n_events'].cumsum()
prod_cart.head()

In [ ]:
# расчитаем кол-во событий за 14 дней лайфтайма
prod_cart_A = round((prod_cart.iloc[-2]['cumsum_n_events'] - prod_cart.iloc[0]['cumsum_n_events']) / prod_cart.iloc[0]['cumsum_n_events'], 2) 
prod_cart_A


In [ ]:
# делаем тоже самое для второй группы
prod_cart_B = round((prod_cart.iloc[-1]['cumsum_n_events'] - prod_cart.iloc[1]['cumsum_n_events']) / prod_cart.iloc[1]['cumsum_n_events'], 2) 
prod_cart_B

Дальше проделаем тоже самое с другим событием - "покупки"

In [ ]:
purchase_page = n_events_lifetime.query('event_name == "purchase"')
purchase_page['cumsum_n_events'] = purchase_page.groupby('group')['n_events'].cumsum()
purchase_page.head()

In [ ]:
# расчитаем кол-во событий за 14 дней лайфтайма
purchase_page_A = round((purchase_page.iloc[-2]['cumsum_n_events'] - purchase_page.iloc[0]['cumsum_n_events']) / purchase_page.iloc[0]['cumsum_n_events'], 2) 
purchase_page_A


In [ ]:
# делаем тоже самое для второй группы
purchase_page_B = round((purchase_page.iloc[-1]['cumsum_n_events'] - purchase_page.iloc[1]['cumsum_n_events']) / purchase_page.iloc[1]['cumsum_n_events'], 2) 
purchase_page_B

**Подвывод:**В целом за 2 недели лайфтайма у пользователей группы А наблюдается больший рост, чем у группы В:

- Просмотр страницы товара с момента регистрации пользователя: А - рост на 13%; B - рост на 20%
- Добавление товара в корзину: А - на 16%; B - на 27%
- Покупка товара: А - на 8%; B - на 12%

## Проверка статистической разницы долей z-критерием.

Нулевая гипотеза: Доли значений метрик в группах равны
<p>
Альтернативная гипотеза: Доли значений метрик в группах не равны

Критический уровень значимости: 0.05. Здесь лучше применить поправку Бонферрони, т.к. один ложноположительный результат может стать проблемой. С поправкой альфа будет равна 0.05 / 3 = 0.016

Из-за того, что кол-во пользователей в группах различно, то нужно проверять z-критерием кол-во событий на одного пользователя.

In [ ]:
alpha = round(0.05 / 3, 3) 
alpha

In [ ]:
#сделаем срез по кол-ву событий в группах и днях жизни
quantity_u = (for_abtest_users.groupby(['lifetime','group'], as_index=False)['user_id'].nunique().rename(columns={'user_id':'n_users'}))
quantity_u

In [ ]:
#события по странице с товаром
prod_page2 = prod_page.merge(quantity_u, on=['lifetime', 'group'])
prod_page2['event_with_u'] = prod_page2['n_events'] / prod_page2['n_users']
prod_page2.head()

In [ ]:
#события по странице с добавлением в корзину товара
prod_cart2 = prod_cart.merge(quantity_u, on=['lifetime', 'group'])
prod_cart2['event_with_u'] = prod_cart2['n_events'] / prod_cart2['n_users']
prod_cart2.head()

In [ ]:
#события покупки
purchase_page2 = purchase_page.merge(quantity_u, on=['lifetime', 'group'])
purchase_page2['event_with_u'] = purchase_page2['n_events'] / purchase_page2['n_users']
purchase_page2.head()

In [ ]:
# посмотрим на средние значения срезов
print('события по странице с товаром')
display(prod_page2.groupby('group')['event_with_u'].mean())
print('Тип события добавление товара в корзину')
display(prod_cart2.groupby('group')['event_with_u'].mean())
print('Тип события покупка')
display(purchase_page2.groupby('group')['event_with_u'].mean())

Выделяется разница у значений события по странице с товаром, остальные группы по оставшимся событиям по этим показателям практически не отличаются

In [ ]:
# создадим функцию для расчета z-test
def z_test(i):
    z_test = ztest(i.query('group == "A"')['event_with_u'], 
                   i.query('group == "B"')['event_with_u'], value=0)[1]
    print("p-value:", z_test)
    if (z_test < alpha):
        print('Отвергаем нулевую гипотезу')  
    else:
        print('Не получилось отвергнуть нулевую гипотезу')
        

In [ ]:
z_test(prod_page2)

In [ ]:
z_test(prod_cart2)

In [ ]:
z_test(purchase_page2)

Ожидаемо получилось, что только у данных групп по стадии просмотра страницы товара p-value оказался меньше, чем альфа. Следовательно, для долей этой метрики - есть значимая разница. Для двух остальных метрик p-value меньше альфы и, следовательно, между долями нет значимой разницы.

## Выводы

После проведения оценки результатов A/B-теста можно сказать следующее:
- на этапе предобработки данных: привели столбцы с датой к нужному типу datetime; дубликаты в данных не были выявлены;
- выяснили, что всего уникальных участников теста:  16666;
- в задании у нас ожидаемое количество участников теста: 6000, но после обработки данных получилось немного больше(6701), но это некритично. А вот уже после дальнейшего анализа выяснилось, что группа А существенно больше (а: 2747; b: 928);
- выяснили, что во время проведения теста шла акция - Christmas&New Year Promo, которая могла оказать существенное влияние, потому что она затрагивает период проведения теста;
- построив график "Распределение числа событий в выборках по дням", выяснили, что: Пик событий 21 декабря и в основном это у группы А; Группа В превосходит группу А только 7 и 9 декабря, а в оставшиеся дни группа А уверенно опережала группу В по кол-ву событий; Начиная c 14 декабря можно наблюдать резкий рост кол-ва событий у группы А;
- сделав воронкa конверсий в выборках на разных этапах, выяснили: Наибольшая просадка коверсии между login и product_page уровнями - (-35% у группы А и -44% у группы В); До покупки доходят 30% пользователей группы А и 28% пользователей группы В; Наименьшая разница в конверсии между product_cart и purchase - (разница в 2% у группы А, у группы В разница и вовсе отсутствует), это свидетельствует о том, что пользователи предпочитают после добавления товара в корзину практически всегда его покупать;
- изучив 2 недели лайфтайма пользователей данных выборок выяснили, что у группы А наблюдается больший рост, чем у группы В: Просмотр страницы товара с момента регистрации пользователя: А - рост на 13%; B - рост на 20%; Добавление товара в корзину: А - на 16%; B - на 27%; Покупка товара: А - на 8%; B - на 12%;
- провели тест "поправку Бонферрони" и установили, что: только у данных групп по стадии просмотра страницы товара p-value оказался меньше, чем альфа. Следовательно, для долей этой метрики - есть значимая разница. Для двух остальных метрик p-value меньше альфы и, следовательно, между долями нет значимой разницы.

**По итогу можно заключить, что нельзя доверять данным этого теста. Множество фактов, которые мы привели в исследовании об этом говорит.** 

Я считаю, что тест засчитывать ни в коем случае нельзя и нужно проводить заново. Следует постараться сделать:
- группы максимально равными по кол-ву участников в них;
- выбрать такие дни проведения теста, чтобы во время них не было акций, которые могли бы исказить результаты;
- чтобы пользователи не участвовали в обоих тестах одновременно;
- соблюдать остальные правила корректного проведения теста.